In [1]:
import numpy as np
import pandas as pd
import _pickle as cPickle
import gzip
from collections import OrderedDict
from stock_helper import *
from stats import *
import stock_helper
import stats
from imp import reload
reload(stock_helper)
reload(stats) # 重新载入模块

<module 'stats' from 'F:\\bigdata\\my_code\\stats.py'>

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import os
CORE_NUM = int(os.environ['NUMBER_OF_PROCESSORS'])
CORE_NUM

8

In [4]:
# 加载数据函数load
def load(path):
    with gzip.open(path, 'rb', compresslevel=1) as file_object:
        raw_data = file_object.read()
    return cPickle.loads(raw_data)

In [5]:
# 保存数据函数save
def save(data, path):
    serialized = cPickle.dumps(data) # 将data对象序列化
    with gzip.open(path, 'wb', compresslevel=1) as file_object: # 以写二进制的模式打开文件
        file_object.write(serialized) # 将序列化的对象写入文件

In [6]:
HEAD_PATH = 'F:/BigData/data'
DATA_PATH = HEAD_PATH + '/stock pkl/'
SAVE_PATH = 'F:/BigData/data'
product_list = ["600276", "600535"] # 防止运行时间过长，只挑选两个股票品种

In [7]:
dire_signal_list = ["nr", "dbook", "range.pos", "price.osci", "ma.dif.10", "kdj.k", "kdj.j"] # 方向性因子
range_signal_list = ["" ,"range", "std", "trend.index"] # 波动性因子
period_list = np.power(2, range(10, 13)) # 周期列表

In [8]:
all_signal_list = np.array([])
for range_signal in range_signal_list:
    for dire_signal in dire_signal_list:
        if len(range_signal)==0:
            signal_name = dire_signal
        else:
            signal_name = dire_signal+"."+range_signal
        all_signal_list = np.append(all_signal_list,signal_name)
signal_list = [signal+".1024" for signal in all_signal_list]

In [9]:
len(signal_list) # 因子总个数

28

In [10]:
signal_list # 所有因子

['nr.1024',
 'dbook.1024',
 'range.pos.1024',
 'price.osci.1024',
 'ma.dif.10.1024',
 'kdj.k.1024',
 'kdj.j.1024',
 'nr.range.1024',
 'dbook.range.1024',
 'range.pos.range.1024',
 'price.osci.range.1024',
 'ma.dif.10.range.1024',
 'kdj.k.range.1024',
 'kdj.j.range.1024',
 'nr.std.1024',
 'dbook.std.1024',
 'range.pos.std.1024',
 'price.osci.std.1024',
 'ma.dif.10.std.1024',
 'kdj.k.std.1024',
 'kdj.j.std.1024',
 'nr.trend.index.1024',
 'dbook.trend.index.1024',
 'range.pos.trend.index.1024',
 'price.osci.trend.index.1024',
 'ma.dif.10.trend.index.1024',
 'kdj.k.trend.index.1024',
 'kdj.j.trend.index.1024']

In [11]:
# 一年的数据里面有多少个4096，向下取整
def count_daily_num(x, period):
    data = load(x)
    return np.floor(np.shape(data)[0]/period)

In [12]:
# 为daily num创建文件夹
for period in period_list:
    os.makedirs(HEAD_PATH+"/daily num "+str(period), exist_ok=True)

In [13]:
# 获取股票数据所有的年份。格式为  “****.pkl”
all_dates = os.listdir(DATA_PATH + product_list[0])
all_dates.sort()
all_dates = np.array(all_dates)
n_days = len(all_dates)

In [14]:
# 针对四个周期得到daily_num
for period in period_list:
    for product in product_list:
        file_list = list(map(lambda x: DATA_PATH+product+"/"+x, all_dates))
        daily_num = pd.Series(parLapply(CORE_NUM, file_list, count_daily_num, period=period))
        save(daily_num, HEAD_PATH+"/daily num "+ str(period) + "/" +product+".pkl")

In [15]:
# 并行生成所有因子的因子分布
def par_get_all_signal(signal_name, file_list, product, period, HEAD_PATH="F:/BigData/data", SAVE_PATH="F:/BigData/data"):
    signal_name = signal_name + "." +str(period)
    n_files = len(file_list)
    all_signal = np.array([])
    for file in file_list:
        S = load(SAVE_PATH+"/tmp pkl/"+product+"/"+signal_name+"/"+file) ## signal
        signal = S
        chosen = (np.arange(len(signal))+1) % period==0
        all_signal = np.concatenate((all_signal, signal[chosen]), axis=0)
    save(all_signal, SAVE_PATH+"/all signal " + str(period) + "/"+product+"."+signal_name+".pkl")

In [16]:
all_signal_list

array(['nr', 'dbook', 'range.pos', 'price.osci', 'ma.dif.10', 'kdj.k',
       'kdj.j', 'nr.range', 'dbook.range', 'range.pos.range',
       'price.osci.range', 'ma.dif.10.range', 'kdj.k.range',
       'kdj.j.range', 'nr.std', 'dbook.std', 'range.pos.std',
       'price.osci.std', 'ma.dif.10.std', 'kdj.k.std', 'kdj.j.std',
       'nr.trend.index', 'dbook.trend.index', 'range.pos.trend.index',
       'price.osci.trend.index', 'ma.dif.10.trend.index',
       'kdj.k.trend.index', 'kdj.j.trend.index'], dtype='<U32')

In [17]:
for period in period_list:
    os.makedirs(HEAD_PATH+"/all signal "+str(period), exist_ok=True)

In [18]:
# 开始并行生成因子分布 包含所有因子，所有周期
for period in period_list:
    for signal_name in all_signal_list:
        for product in product_list:
            parLapply(CORE_NUM, [signal_name], par_get_all_signal, file_list=all_dates, product=product, period=period, SAVE_PATH=SAVE_PATH)

In [19]:
# 利用抽样因子形成训练和测试矩阵
def get_sample_signal(good_night_files, sample, product, signal_list, period, daily_num):
    n_samples = sum(daily_num[sample]) # 样本总数
    n_signal = len(signal_list) # 因子数量, 因子矩阵应该是 n_samples*n_signal 大小
    all_signal = np.ndarray(shape=(int(n_samples),n_signal))
    cur = 0
    for file in good_night_files[sample]:
        data = load(HEAD_PATH+"/stock pkl/"+product+"/"+file)
        chosen = (np.arange(np.shape(data)[0])+1) % period==0
        n_chosen = sum(chosen)
        for i in range(n_signal):
            signal_name = signal_list[i]
            S = load(SAVE_PATH+"/tmp pkl/"+product+"/"+signal_name+"/"+file)
            signal = S[(np.arange(len(S))+1) % period == 0]
            signal[np.isnan(signal)] = 0 # 去除缺省值
            signal[np.isinf(signal)] = 0 # 去除无穷值
            all_signal[cur:(cur+n_chosen),i] = signal
        cur = cur+n_chosen
    all_signal = pd.DataFrame(all_signal, columns=signal_list)
    return all_signal

In [20]:
for period in period_list:
    os.makedirs(HEAD_PATH+"/train test mat "+str(period), exist_ok=True)

In [21]:
# 布尔索引
train_sample = (all_dates<"2019")
test_sample = (all_dates>"2019")

In [22]:
# 获得各个周期的训练集矩阵
for period in period_list:
    print(period)
    signal_list = [signal+"."+str(period) for signal in all_signal_list]
    for product in product_list:
        print(product)
        daily_num = load(HEAD_PATH+"/daily num " + str(period) + "/"+product+".pkl")
        train_mat = get_sample_signal(all_dates, train_sample, product, 
                                      np.append(signal_list, ["ret."+str(period),  "ret."+str(period)+".001","ret."+str(period)+".002","ret."+str(period)+".004" ]), 
                                      period, daily_num)
        save(train_mat, HEAD_PATH+"/train test mat " + str(period) + "/"+product+".train.mat.pkl")

1024
600276
600535
2048
600276
600535
4096
600276
600535


In [23]:
# 获得各个周期的测试集矩阵
for period in period_list:
    print(period)
    signal_list = [signal+"."+str(period) for signal in all_signal_list]
    for product in product_list:
        print(product)
        daily_num = load(HEAD_PATH+"/daily num " + str(period) + "/"+product+".pkl")
        train_mat = get_sample_signal(all_dates, test_sample, product, 
                                      np.append(signal_list, ["ret."+str(period),  "ret."+str(period)+".001","ret."+str(period)+".002","ret."+str(period)+".004" ]), 
                                      period, daily_num)
        save(train_mat, HEAD_PATH+"/train test mat " + str(period) + "/"+product+".test.mat.pkl")

1024
600276
600535
2048
600276
600535
4096
600276
600535


In [24]:
ori_mat = load(HEAD_PATH+"/train test mat 1024/"+product_list[0]+".test.mat.pkl")
ori_mat

,nr.1024,dbook.1024,range.pos.1024,price.osci.1024,ma.dif.10.1024,kdj.k.1024,kdj.j.1024,nr.range.1024,dbook.range.1024,range.pos.range.1024,...,dbook.trend.index.1024,range.pos.trend.index.1024,price.osci.trend.index.1024,ma.dif.10.trend.index.1024,kdj.k.trend.index.1024,kdj.j.trend.index.1024,ret.1024,ret.1024.001,ret.1024.002,ret.1024.004
0,-0.016187,-0.032347,-0.500000,0.000000,-0.003059,-1.000000,-1.000000,-0.000000,-0.000000,-0.000000,...,-0.000000,-0.000000,0.000000,-0.000000,-0.000000,-0.000000,0.009362,0.009362,0.009362,0.009362
1,0.029679,-0.087331,-0.144546,0.166250,0.002561,0.379399,-0.052040,0.059684,-0.175621,-0.290680,...,-0.040730,-0.067414,0.077536,0.001194,0.176945,-0.024271,-0.008223,-0.008223,-0.008223,-0.008223
2,-0.004971,-0.165667,-0.132470,-0.103364,-0.001604,-0.469673,-0.225817,-0.007000,-0.233309,-0.186558,...,-0.096965,-0.077535,-0.060499,-0.000939,-0.274899,-0.132171,0.005288,0.005288,0.005288,0.005288
3,0.014578,-0.191106,0.028367,0.167863,0.002295,0.397458,0.315343,0.014655,-0.192108,0.028515,...,-0.100624,0.014936,0.088386,0.001208,0.209275,0.166039,-0.037451,-0.011000,-0.021000,-0.037451
4,-0.068987,-0.124890,-0.161964,-0.395623,-0.022957,-0.557487,-0.579179,-0.382179,-0.691876,-0.897263,...,-0.083162,-0.107849,-0.263439,-0.015287,-0.371222,-0.385666,-0.012312,-0.011000,-0.012312,-0.012312
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1904,0.054391,-0.030042,0.129948,0.461284,0.003961,0.153857,0.019984,0.134093,-0.074065,0.320369,...,-0.009976,0.043150,0.153170,0.001315,0.051088,0.006636,0.013070,0.011000,0.013070,0.013070
1905,0.081426,-0.008600,0.285489,0.601259,0.011139,0.902649,0.781896,0.337617,-0.035657,1.183729,...,-0.005610,0.186251,0.392258,0.007267,0.588882,0.510103,-0.013187,-0.011000,-0.013187,-0.013187
1906,-0.025570,-0.089672,0.053757,-0.161285,-0.001164,-0.691358,-0.220118,-0.079037,-0.277176,0.166164,...,-0.079173,0.047463,-0.142401,-0.001028,-0.610410,-0.194345,-0.003750,-0.003750,-0.003750,-0.003750
1907,-0.036547,-0.009712,-0.133084,-0.533156,-0.004830,-0.631787,-0.642927,-0.073344,-0.019490,-0.267076,...,-0.003724,-0.051029,-0.204431,-0.001852,-0.242250,-0.246521,-0.000535,-0.000535,-0.000535,-0.000535


In [25]:
ori_mat = load(HEAD_PATH+"/train test mat 2048/"+product_list[0]+".test.mat.pkl")
ori_mat

,nr.2048,dbook.2048,range.pos.2048,price.osci.2048,ma.dif.10.2048,kdj.k.2048,kdj.j.2048,nr.range.2048,dbook.range.2048,range.pos.range.2048,...,dbook.trend.index.2048,range.pos.trend.index.2048,price.osci.trend.index.2048,ma.dif.10.trend.index.2048,kdj.k.trend.index.2048,kdj.j.trend.index.2048,ret.2048,ret.2048.001,ret.2048.002,ret.2048.004
0,0.020084,-0.080120,-0.500000,0.000000,0.000931,-1.000000,-1.000000,0.000000,-0.000000,-0.000000,...,-0.000000,-0.000000,0.000000,0.000000,-0.000000,-0.000000,-0.002935,-0.002935,-0.002935,-0.002935
1,0.008950,-0.174008,-0.125712,0.183950,0.002273,0.110308,-0.059953,0.012604,-0.245055,-0.177040,...,-0.036448,-0.026332,0.038530,0.000476,0.023105,-0.012558,-0.049762,-0.011000,-0.021000,-0.041000
2,-0.058852,-0.114084,-0.206563,-0.431840,-0.022548,-0.509685,-0.420298,-0.326031,-0.632013,-1.144340,...,-0.100325,-0.181651,-0.379759,-0.019829,-0.448216,-0.369609,-0.002205,-0.002205,-0.002205,-0.002205
3,-0.030417,-0.038606,-0.177970,-0.699374,-0.012983,-0.048851,-0.356446,-0.053019,-0.067293,-0.310211,...,-0.004659,-0.021476,-0.084396,-0.001567,-0.005895,-0.043014,-0.001070,-0.001070,-0.001070,-0.001070
4,-0.016978,-0.058539,-0.017550,-0.189618,-0.004813,0.427363,0.234135,-0.043605,-0.150344,-0.045072,...,-0.002322,-0.000696,-0.007523,-0.000191,0.016955,0.009289,0.045311,0.011000,0.021000,0.041000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
949,0.022869,-0.050831,0.020133,0.429086,0.005582,0.283172,-0.001293,0.073516,-0.163407,0.064722,...,-0.027314,0.010819,0.230569,0.003000,0.152162,-0.000695,0.002881,0.002881,0.002881,0.002881
950,0.020064,-0.054645,0.199987,0.989974,0.006684,0.649674,0.731282,0.026503,-0.072182,0.264168,...,-0.024243,0.088723,0.439194,0.002965,0.288223,0.324428,0.009195,0.009195,0.009195,0.009195
951,0.038167,-0.040078,0.220885,0.617268,0.006221,0.421325,0.504569,0.094096,-0.098807,0.544559,...,-0.030593,0.168608,0.471178,0.004749,0.321610,0.385152,-0.000117,-0.000117,-0.000117,-0.000117
952,0.005510,-0.066648,0.262288,0.191261,0.006020,0.367260,0.652619,0.024075,-0.291219,1.146076,...,-0.000367,0.001444,0.001053,0.000033,0.002022,0.003593,-0.004284,-0.004284,-0.004284,-0.004284


# 下面是一些写得很失败的代码

In [26]:
data_600276_2016 = pd.DataFrame()

In [27]:
data_600276_2016 # 因子矩阵未加入数据为空

""


In [28]:
data1 = load(HEAD_PATH+'/tmp pkl/'+product_list[0]+'/nr.1024/'+'2016.pkl')
data2 = load(HEAD_PATH+'/tmp pkl/'+product_list[0]+'/dbook.1024/'+'2016.pkl')

In [29]:
len(data1)

646835

In [30]:
len(data2)

646835

In [31]:
data_600276_2016['nr.1024'] = data1
data_600276_2016['dbook.1024'] = data2

In [32]:
data_600276_2016

,nr.1024,dbook.1024
0,0.000000,0.000000
1,1.000000,0.500169
2,-0.958692,0.333333
3,-0.446148,0.249915
4,-0.562882,0.199865
...,...,...
646830,0.010561,-0.226725
646831,0.010502,-0.227925
646832,0.010491,-0.228447
646833,0.012744,-0.228969


In [33]:
data_600276_2016['nr.1024']

0         0.000000
1         1.000000
2        -0.958692
3        -0.446148
4        -0.562882
            ...   
646830    0.010561
646831    0.010502
646832    0.010491
646833    0.012744
646834    0.012744
Name: nr.1024, Length: 646835, dtype: float64

In [34]:
for signal in signal_list:
    path = HEAD_PATH+'/tmp pkl/'+product_list[0]+'/'+signal+'/'+'2016.pkl'
    data = load(path)
    data_600276_2016[signal] = data

In [35]:
data_test_x = data_600276_2016[100000:600000] # 因子矩阵
data_test_x.reset_index(drop=True)

,nr.1024,dbook.1024,nr.4096,dbook.4096,range.pos.4096,price.osci.4096,ma.dif.10.4096,kdj.k.4096,kdj.j.4096,nr.range.4096,...,ma.dif.10.std.4096,kdj.k.std.4096,kdj.j.std.4096,nr.trend.index.4096,dbook.trend.index.4096,range.pos.trend.index.4096,price.osci.trend.index.4096,ma.dif.10.trend.index.4096,kdj.k.trend.index.4096,kdj.j.trend.index.4096
0,0.004813,0.030213,-0.001771,-0.029704,-0.016480,-0.120537,-0.006571,-0.347675,-0.306877,-0.003402,...,-0.003333,-0.176369,-0.155673,-0.000797,-0.013364,-0.007415,-0.054233,-0.002957,-0.156427,-0.138071
1,0.004798,0.030192,-0.001775,-0.029698,-0.016475,-0.120306,-0.006564,-0.347368,-0.306911,-0.003409,...,-0.003329,-0.176169,-0.155651,-0.000799,-0.013365,-0.007414,-0.054141,-0.002954,-0.156326,-0.138119
2,0.005113,0.030849,-0.001692,-0.029524,-0.016471,-0.120068,-0.006555,-0.347057,-0.306945,-0.003249,...,-0.003324,-0.175967,-0.155630,-0.000757,-0.013214,-0.007372,-0.053737,-0.002934,-0.155327,-0.137375
3,0.005783,0.029474,-0.001514,-0.029858,-0.016465,-0.119813,-0.006546,-0.346737,-0.306979,-0.002909,...,-0.003318,-0.175761,-0.155608,-0.000670,-0.013205,-0.007282,-0.052989,-0.002895,-0.153350,-0.135766
4,0.008226,0.030131,-0.000867,-0.029683,-0.016457,-0.119498,-0.006531,-0.346385,-0.307012,-0.001665,...,-0.003310,-0.175538,-0.155585,-0.000367,-0.012557,-0.006962,-0.050551,-0.002763,-0.146532,-0.129876
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,-0.034104,-0.196097,0.002131,-0.212613,0.114097,0.248467,0.005148,-0.143209,0.139442,0.001740,...,0.000883,-0.024560,0.023914,0.001009,-0.100647,0.054011,0.117620,0.002437,-0.067793,0.066009
499996,-0.034543,-0.197318,0.002025,-0.212915,0.114004,0.247886,0.005142,-0.143825,0.139202,0.001654,...,0.000882,-0.024672,0.023879,0.001001,-0.105228,0.056344,0.122512,0.002541,-0.071082,0.068797
499997,-0.033007,-0.195831,0.002369,-0.212541,0.113913,0.247353,0.005134,-0.144416,0.138962,0.001935,...,0.000881,-0.024779,0.023843,0.001120,-0.100427,0.053825,0.116876,0.002426,-0.068238,0.065661
499998,-0.033610,-0.194345,0.002224,-0.212166,0.113822,0.246803,0.005128,-0.145017,0.138722,0.001816,...,0.000880,-0.024888,0.023808,0.001070,-0.102037,0.054740,0.118694,0.002466,-0.069742,0.066715


In [36]:
X = np.array(data_test_x)
X # 自变量矩阵

array([[ 0.00481266,  0.03021277, -0.00177114, ..., -0.00295655,
        -0.15642743, -0.13807118],
       [ 0.0047981 ,  0.03019233, -0.00177494, ..., -0.00295382,
        -0.15632573, -0.13811889],
       [ 0.00511275,  0.03084857, -0.00169172, ..., -0.0029338 ,
        -0.15532746, -0.13737515],
       ...,
       [-0.03300687, -0.19583126,  0.00236949, ...,  0.00242591,
        -0.06823765,  0.06566065],
       [-0.03360992, -0.1943454 ,  0.00222419, ...,  0.00246611,
        -0.06974245,  0.06671509],
       [-0.03361381, -0.19421389,  0.00222325, ...,  0.00235014,
        -0.06682558,  0.06355093]])

In [37]:
data_test_y = load('F:/BigData/data/tmp pkl/600276/ret.1024.004/2016.pkl')[100000:600000]
data_test_y

array([-8.46898229e-03, -8.39838780e-03, -9.34663870e-03, ...,
        1.30613784e-05,  6.62472250e-05,  3.01214359e-05])

In [38]:
len(data_test_y)

500000

In [39]:
y = np.array(data_test_y)
y # 因变量

array([-8.46898229e-03, -8.39838780e-03, -9.34663870e-03, ...,
        1.30613784e-05,  6.62472250e-05,  3.01214359e-05])

In [40]:
import mglearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso, LassoCV

In [41]:
# 训练集和数据集划分用于交叉验证
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=42, test_size=0.1)

In [42]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)
# 对特征标准化，不需要对标签标准化

In [43]:
X_train.shape

(450000, 30)

In [44]:
X_train

array([[-0.22489623, -0.83654088, -0.19531336, ..., -0.18021805,
        -0.08378792, -0.14363915],
       [-0.94319504,  0.46872728, -1.16367679, ..., -1.30344608,
        -1.60303785, -1.48647827],
       [ 2.32516151, -2.3918466 ,  2.21760782, ...,  1.86126589,
         2.01348258,  1.98823735],
       ...,
       [-0.50587986,  1.42532354, -0.50553188, ..., -0.32219835,
        -0.19364041,  0.01848402],
       [-1.50973934,  0.4495888 , -1.90323438, ..., -1.5435234 ,
        -1.53130018, -1.8253906 ],
       [-0.80928653,  0.78260349, -0.27417167, ...,  0.03712489,
        -1.48600554, -0.90721476]])

In [45]:
y_train.shape

(450000,)

In [46]:
X_test.shape

(50000, 30)

In [47]:
y_test.shape

(50000,)

In [48]:
y_std = np.std(y_train) # 求标准差
y_std_test = np.std(y_test)

In [49]:
scaler =  StandardScaler(copy=True, with_mean=True, with_std=True)
scaler.fit(X_train)
X_std = np.sqrt(scaler.var_)

# Lasso模型

In [50]:
lasso = Lasso(alpha=0.001,max_iter=100).fit(X_train,y_train/y_std)

In [51]:
print("lasso.coef_:{}".format(lasso.coef_/X_std*y_std)) # 系数，有几个自变量就有几个系数
print("lasso.intercept_:{}".format(lasso.intercept_)) # 截距

lasso.coef_:[-6.79234194e-04  1.86939652e-04  1.31283551e-03  0.00000000e+00
 -3.80265660e-04  1.08879293e-04 -3.23270622e-03  4.33560357e-05
 -2.11879575e-03 -1.25717244e-03 -1.32511465e-04  4.15211502e-04
  5.23613099e-04 -0.00000000e+00 -8.52284553e-04  2.78626177e-03
 -3.91973146e-04  7.88463790e-04  4.74246058e-04  5.74984268e-04
  8.44101709e-04 -0.00000000e+00  7.29362392e-04 -4.29159545e-04
 -7.62417586e-04 -8.89567551e-04  2.75263044e-03 -1.03783172e-03
  8.90460004e-04 -1.10828829e-03]
lasso.intercept_:0.05880972877158159


In [52]:
print("Traing set score: {:.2f}".format(lasso.score(X_train,y_train/y_std))) # 训练集拟合优度
print("Test set score:{:.2f}".format(lasso.score(X_test,y_test/y_std_test))) # 测试集拟合优度

Traing set score: 0.03
Test set score:0.03


# LassoCV

In [ ]:
# 自带交叉检验
model = LassoCV(n_alphas=100, fit_intercept=False, cv=5, max_iter=10000).fit(X_train,y_train/y_std)

In [ ]:
print("lasso.coef_:{}".format(model.coef_/X_std*y_std)) # 系数，有几个自变量就有几个系数

In [ ]:
print("Traing set score: {:.2f}".format(lasso.score(X_train,y_train/y_std))) # 训练集拟合优度
print("Test set score:{:.2f}".format(lasso.score(X_test,y_test/y_std_test))) # 测试集拟合优度